<h1 style="text-align:center;">Text Analysis - Assignment</h1><hr>

## Dependencies :

In [24]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import openpyxl
import csv
import itertools
import os
from nltk import word_tokenize , sent_tokenize 
import re

## Extracting Text AND Saving into file :

In [19]:
input_dir="Input (1).xlsx"
df=pd.read_excel(input_dir) #reading excel file
URL_ID=df['URL_ID'].to_list()       #getting particluar columns' content in a list 
URL_LIST=df['URL'].to_list()

def extract_data(URL): #to scrape data from Webpages
    article=''
    try:
        response=requests.get(URL) # making get request
        soup = BeautifulSoup(response.content, 'html.parser')  #creating a soup object with response data
        title=soup.find('h1').get_text() #getting title of webpage only
        
        content=soup.find_all(attrs={"class":"td-post-content"}) #getting only the required post content
        for i in content:
           article+= i.get_text()
        text = title + '\n' + article #merging title and text
        return text
    except:
         print(f'Failed to get response from URL: {URL}' )
        
def save_text(URL , ID): # to save scraped data into files
        try:
            text = extract_data(URL)
            file_name= ID +'.txt'
            with open(file_name , 'w' ,encoding="utf-8") as f: #file if not present will be created with the file name as URL ID
                f.write(text) 
        except:
            print('Unable to write to file')

## Data Analysis :

### 1. Sentimental Analysis :

#### 1.1 Cleaning using Stop Words

In [3]:
stword_dir='StopWords-20240505T100503Z-001/StopWords' #To open the directory to acccess files with stop words

stop_words=[]                                             #All stop words will be included in this list
for file in os.listdir(stword_dir):                       #iterating over list of files in particular directory
    file_name = os.path.join(stword_dir, file)            #concatinating file name with the directory name
    with open(file_name , 'r')as f: 
        words_list=f.read().replace('|','\n').split("\n") #splitting string data into list of words in particular file
        stop_words+=words_list                            #merging all lists of stopwords into a single list.

def clean_text(tokenised_list):
    cleaned_words=[word for word in tokenised_list if word.casefold() not in stop_words]   #filtering out all lowercase stopwords
    cleaned_words=[word for word in cleaned_words if word.upper() not in stop_words]       #filtering out all uppercase stop words
    return cleaned_words

##### WORD COUNT :

In [25]:
                       #     ---Calculated Inside main() function---


# word_tokens=word_tokenize(no_punct)   #Words list without punctuations
# cleaned_words=clean_text(word_tokens) #Words list without punctuations and stop words
# sent_tokens=sent_tokenize(textfile)   #List of sentences in the text


# Num_clean_words=len(cleaned_words)
# Num_sent=len(sent_tokens)


#### 1.2 Postive and Negative words :

In [5]:
Ngword_dir='MasterDictionary-20240505T100418Z-001/MasterDictionary/negative-words.txt'
Psword_dir='MasterDictionary-20240505T100418Z-001/MasterDictionary/positive-words.txt'

with open(Psword_dir , 'r') as f:
    Ps_words=f.read().split('\n')    #Creating positive words list
    Pstv_words=clean_text(Ps_words)  #Filtering out stopwords
    
with open(Ngword_dir , 'r') as f:    #Creating negative words list
    Ng_words=f.read().split('\n')    #filtering out stopwords
    Ngtv_words=clean_text(Ng_words)


#### 1.3 Calculating Derived Variables :

##### POSITIVE SCORE

In [6]:
def positive_score(filtered_list):
    Pstv_score=0                          #initialising score
    for word in filtered_list:
        if word.casefold() in Pstv_words: 
            Pstv_score+=1                 #increment score +1 if word found
    return Pstv_score

##### NEGATIVE SCORE

In [7]:
def negative_score(filtered_list):
    Ngtv_score=0
    for word in filtered_list:
        if word.casefold() in Ngtv_words:
            Ngtv_score-=1                   #increment score by -1 if word found
    return -(Ngtv_score)                    #returning positive value of score


##### POLARITY SCORE

In [8]:
def polarity_score(ps_score, ng_score):
    try:
        Polarity_Score = (ps_score - ng_score)/ ((ps_score+ ng_score) + 0.000001)
        return Polarity_Score
    except ZeroDivisionError:
        return 0

##### SUBJECTIVITY SCORE

In [9]:
def subjectivity_score(ps_score, ng_score,cleaned_words):
    try:
        Subjectivity_Score = (ps_score + ng_score)/ ((len(cleaned_words)) + 0.000001)
        return Subjectivity_Score
    except ZeroDivisionError:
        return 0

##### SYLLABLE COUNT PER WORD AND COMPLEX WORDS COUNT :

In [10]:
def syllable_count(word): #to count number of syllables in a word.
    vowel='aeiou'
    not_count=['ed','es']
    count=0
    word=word.casefold()
    if word[-2:] not in not_count:  
        for index in range(0,len(word)):
            if word[index] in vowel:
                count+=1
    else:
        for index in range(0,len(word)): 
            if word[index] in vowel:
                count+=1
        count=count-1      #not counting 'ed' and 'es' as a syllable.
    return count

def total_syllable_count(words_list):
    total_count=0
    for word in words_list:
        count=syllable_count(word)
        total_count+=count
    return total_count

def complex_words_count(words_list):
    complex_words=[word for word in words_list if syllable_count(word) > 2]
    cmplx_count=len(complex_words)
    return cmplx_count

##### AVERAGE SENTENCE LENGTH :

In [11]:
def avg_sent_len(Num_clean_words,Num_sent):
    try:
        avg=Num_clean_words/Num_sent
        return avg
    except ZeroDivisionError:
        return 0

##### PERCENTAGE COMPLEX WORDS :

In [12]:
def per_cmplx(cmplx_count,Num_clean_words):
    try:
        per=(cmplx_count/Num_clean_words)*100
        return per
    except ZeroDivisionError:
        return 0

##### FOG INDEX :

In [13]:
def fog_index(per_cmplx,avg_sent_len):
    fog_index=0.4*(avg_sent_len + per_cmplx)
    return fog_index

##### PERSONAL PRONOUNS :

In [14]:
def pers_pronoun(words_list):
    pronouns=['I' , 'we' ,'We','My', 'my' , 'Ours','ours' , 'us' , 'Us' ]
    count=0
    for word in words_list:
        if word in pronouns:
            count+=1
    return count

##### AVERGAE WORD LENGTH :

In [15]:
def total_chars(words_list):
    total_chars=0
    for word in words_list:
        total_chars+=len(word)
    return total_chars
def avg_word_len(cleaned_words ,Num_clean_words):
    try:
        Num_chars=total_chars(cleaned_words)
        Avg_word_len = Num_chars/Num_clean_words
        return Avg_word_len
    except ZeroDivisionError:
        return 0

## MAIN FUNCTION :

In [18]:
def main(): 
    Output_data=[]
    for URL , ID in zip(URL_LIST,URL_ID): #using zip to iterate over two lists simultaneously
        save_text(URL , ID)    #extract from url and save text to file
        file_name= ID + '.txt'
        
        with open(file_name,'r',encoding="utf8") as f:
            textfile=f.read()
        no_punct = re.sub(r'[^\w\s]', '', textfile) #removing all punctuations  from text

        word_tokens = word_tokenize(no_punct)   #Words list without punctuations
        cleaned_words = clean_text(word_tokens) #Words list without punctuations and stop words
        sent_tokens = sent_tokenize(textfile)   #List of sentences in the text
        
        
        Num_clean_words = len(cleaned_words)    #Number of cleaned words
        Num_sent = len(sent_tokens)             #Number of sentences

        Positive_Score = positive_score(cleaned_words)                                #Computing Positive Score
        Negative_Score = negative_score(cleaned_words)                                #Computing Negative Score
        Polarity_Score = polarity_score(Positive_Score ,Negative_Score)               #Computing Polarity Score
        Subjectivity_Score = subjectivity_score(Positive_Score ,Negative_Score ,cleaned_words)#Computing Subjectivity Score
        cmplx_count = complex_words_count(cleaned_words)                              #Computing No. of Complex Words
        Avg_sent_len = avg_sent_len(Num_clean_words,Num_sent)                                    #Average Sentence Length
        Per_cmplx = per_cmplx(cmplx_count,Num_clean_words)                                      #Computing Percentage of Complex Words
        Fog_index = fog_index(Per_cmplx,Avg_sent_len)                                          #Computing Fog Index
        Pers_pronoun = pers_pronoun(word_tokens)                                              #Computing No. of Personal Pronouns
        Avg_word_len = avg_word_len(cleaned_words,Num_clean_words)                           #Computing average word length
        Syllable_per_word = total_syllable_count(cleaned_words)                             #Computing syllables per word
        
        #Append output to the Output file
        Data ={'URL_ID':ID, 'URL':URL,'POSITIVE SCORE':Positive_Score, 'NEGATIVE SCORE': Negative_Score, 'POLARITY SCORE':Polarity_Score ,
       'SUBJECTIVITY SCORE':Subjectivity_Score, 'AVG SENTENCE LENGTH':Avg_sent_len,
       'PERCENTAGE OF COMPLEX WORDS':Per_cmplx, 'FOG INDEX':Fog_index,
       'AVG NUMBER OF WORDS PER SENTENCE':Avg_sent_len, 'COMPLEX WORD COUNT':cmplx_count, 'WORD COUNT':Num_clean_words,
       'SYLLABLE PER WORD':Syllable_per_word, 'PERSONAL PRONOUNS':Pers_pronoun, 'AVG WORD LENGTH':Avg_word_len}
        Output_data.append(Data)  #appending rows in the form of dictionaries to a list
    df = pd.DataFrame.from_records(Output_data) #creating a dataframe from list of rows as dictionaries
    df.to_excel('output.xlsx' , index=False)  #creating excel file from DataFrame
 

## RUN Programme :

In [ ]:
if __name__ == "__main__":
    main()   

In [21]:
!pip freeze > requirements.txt